In [1]:
from buildstock_query import BuildStockQuery, ExeId, MappedColumn
import pandas as pd
import sqlalchemy as sa

In [4]:
my_run = BuildStockQuery('rescore','euss-oedi',
                                "resstock_amy2018_release_2022_1",
                                db_schema="resstock_oedi",
                                skip_reports=False)

INFO:buildstock_query.query_core:Loading resstock_amy2018_release_2022_1 ...
INFO:botocore.tokens:Loading cached SSO token for nrel-sso


SELECT resstock_amy2018_release_2022_1_metadata.applicability, resstock_amy2018_release_2022_1_metadata."in.sqft", resstock_amy2018_release_2022_1_metadata.weight, resstock_amy2018_release_2022_1_metadata."in.ahs_region", resstock_amy2018_release_2022_1_metadata."in.ashrae_iecc_climate_zone_2004", resstock_amy2018_release_2022_1_metadata."in.ashrae_iecc_climate_zone_2004_2_a_split", resstock_amy2018_release_2022_1_metadata."in.bathroom_spot_vent_hour", resstock_amy2018_release_2022_1_metadata."in.bedrooms", resstock_amy2018_release_2022_1_metadata."in.building_america_climate_zone", resstock_amy2018_release_2022_1_metadata."in.cec_climate_zone", resstock_amy2018_release_2022_1_metadata."in.ceiling_fan", resstock_amy2018_release_2022_1_metadata."in.census_division", resstock_amy2018_release_2022_1_metadata."in.census_division_recs", resstock_amy2018_release_2022_1_metadata."in.census_region", resstock_amy2018_release_2022_1_metadata."in.city", resstock_amy2018_release_2022_1_metadata."i

INFO:buildstock_query.query_core:17 queries cache read from None.
INFO:buildstock_query.query_core:17 queries cache is updated.
INFO:buildstock_query.main:Getting Success counts...
INFO:buildstock_query.report_query:Checking integrity with ts_tables ...


          success  inapplicable  fail       Sum  Applied %  no-chng  bad-chng  \
upgrade                                                                         
0        548916.0           0.0     0  548916.0        0.0        0         0   
1        531823.0       17093.0     0  548916.0       96.9        0         0   
10       548916.0           0.0     0  548916.0      100.0        0         0   
2        536797.0       12119.0     0  548916.0       97.8        0         0   
3        548260.0         656.0     0  548916.0       99.9        0         0   
4        548260.0         656.0     0  548916.0       99.9        0         0   
5        532156.0       16760.0     0  548916.0       96.9        0         0   
6        545331.0        3585.0     0  548916.0       99.3        0         0   
7        400579.0      148337.0     0  548916.0       73.0        0         0   
8        548913.0           3.0     0  548916.0      100.0        0         0   
9        548916.0           

INFO:buildstock_query.query_core:No new queries to save.


Annual and timeseries tables are verified to have the same number of buildings.
All buildings are verified to have the same number of (35040) timeseries rows.


In [3]:
query = my_run.get_building_ids(get_query_only=True)
print(query)

SELECT baseline.bldg_id 
FROM (SELECT resstock_amy2018_release_2022_1_metadata.applicability AS applicability, resstock_amy2018_release_2022_1_metadata."in.sqft" AS "in.sqft", resstock_amy2018_release_2022_1_metadata.weight AS weight, resstock_amy2018_release_2022_1_metadata."in.ahs_region" AS "in.ahs_region", resstock_amy2018_release_2022_1_metadata."in.ashrae_iecc_climate_zone_2004" AS "in.ashrae_iecc_climate_zone_2004", resstock_amy2018_release_2022_1_metadata."in.ashrae_iecc_climate_zone_2004_2_a_split" AS "in.ashrae_iecc_climate_zone_2004_2_a_split", resstock_amy2018_release_2022_1_metadata."in.bathroom_spot_vent_hour" AS "in.bathroom_spot_vent_hour", resstock_amy2018_release_2022_1_metadata."in.bedrooms" AS "in.bedrooms", resstock_amy2018_release_2022_1_metadata."in.building_america_climate_zone" AS "in.building_america_climate_zone", resstock_amy2018_release_2022_1_metadata."in.cec_climate_zone" AS "in.cec_climate_zone", resstock_amy2018_release_2022_1_metadata."in.ceiling_fan" 

In [4]:
building_type_map = {"Mobile Home": "MH", "Single-Family Detached": "SF",
                     "Single-Family Attached": "SF", "Multi-Family with 2 - 4 Units": "MF",
                     "Multi-Family with 5+ Units": "MF",}
bldg_col = my_run._get_column('in.geometry_building_type_recs')
simple_bldg_col = MappedColumn(bsq=my_run, name='simple_bldg_type', mapping_dict=building_type_map,
                               key=bldg_col)
mapped_agg_query = my_run.agg.aggregate_annual(enduses=['electricity.total.energy_consumption'],
                                       group_by=[simple_bldg_col],
                                          get_query_only=False)
print(mapped_agg_query)

INFO:botocore.tokens:Loading cached SSO token for nrel-sso


  simple_bldg_type  sample_count   units_count  \
0               MF        144213  3.491844e+07   
1               MH         34491  8.351341e+06   
2               SF        370212  8.963981e+07   

   electricity.total.energy_consumption  
0                          2.632233e+11  
1                          9.934359e+10  
2                          1.188864e+12  


In [5]:
dryer_co2_impact = {"Gas, 120% Usage": 1.2 * 1.5, 
                    "Gas, 100% Usage": 1 * 1.5,
                    "Gas, 80% Usage": 0.8 * 1.5,
                    "Electric, 120% Usage": 1.2 * 0.5,
                    "Electric 100% Usage": 1 * 0.5,
                    "Electric 80% Usage": 0.8 * 0.5,
                    "None": 0}
bldg_col = my_run._get_column('in.clothes_dryer')
impact_col = MappedColumn(bsq=my_run, name='dryer_impact', mapping_dict=dryer_co2_impact,
                               key=bldg_col)
mapped_agg_query = my_run.query(enduses=[impact_col],
                                               get_query_only=False)
print(mapped_agg_query)

   sample_count   units_count  dryer_impact
0        548916  1.329096e+08  4.579291e+07


In [11]:
baseline_agg_query = my_run.query(enduses=['electricity.total.energy_consumption'],
                                       group_by=['geometry_building_type_recs'],
                                          get_query_only=True,
                                          )
print(baseline_agg_query)



SELECT baseline."in.geometry_building_type_recs" AS geometry_building_type_recs, sum(1) AS sample_count, sum(baseline.weight) AS units_count, sum(baseline."out.electricity.total.energy_consumption" * baseline.weight) AS "electricity.total.energy_consumption" 
FROM (SELECT resstock_amy2018_release_2022_1_metadata.applicability AS applicability, resstock_amy2018_release_2022_1_metadata."in.sqft" AS "in.sqft", resstock_amy2018_release_2022_1_metadata.weight AS weight, resstock_amy2018_release_2022_1_metadata."in.ahs_region" AS "in.ahs_region", resstock_amy2018_release_2022_1_metadata."in.ashrae_iecc_climate_zone_2004" AS "in.ashrae_iecc_climate_zone_2004", resstock_amy2018_release_2022_1_metadata."in.ashrae_iecc_climate_zone_2004_2_a_split" AS "in.ashrae_iecc_climate_zone_2004_2_a_split", resstock_amy2018_release_2022_1_metadata."in.bathroom_spot_vent_hour" AS "in.bathroom_spot_vent_hour", resstock_amy2018_release_2022_1_metadata."in.bedrooms" AS "in.bedrooms", resstock_amy2018_release_20

In [12]:
baseline_agg = my_run.execute("""
SELECT baseline."in.geometry_building_type_recs" AS geometry_building_type_recs, sum(1) AS sample_count, sum(baseline.weight) AS units_count, sum(baseline."out.electricity.total.energy_consumption" * baseline.weight) AS electricitytotalenergy_consumption 
FROM (SELECT resstock_amy2018_release_2022_1_metadata.applicability AS applicability, resstock_amy2018_release_2022_1_metadata."in.sqft" AS "in.sqft", resstock_amy2018_release_2022_1_metadata.weight AS weight, resstock_amy2018_release_2022_1_metadata."in.ahs_region" AS "in.ahs_region", resstock_amy2018_release_2022_1_metadata."in.ashrae_iecc_climate_zone_2004" AS "in.ashrae_iecc_climate_zone_2004", resstock_amy2018_release_2022_1_metadata."in.ashrae_iecc_climate_zone_2004_2_a_split" AS "in.ashrae_iecc_climate_zone_2004_2_a_split", resstock_amy2018_release_2022_1_metadata."in.bathroom_spot_vent_hour" AS "in.bathroom_spot_vent_hour", resstock_amy2018_release_2022_1_metadata."in.bedrooms" AS "in.bedrooms", resstock_amy2018_release_2022_1_metadata."in.building_america_climate_zone" AS "in.building_america_climate_zone", resstock_amy2018_release_2022_1_metadata."in.cec_climate_zone" AS "in.cec_climate_zone", resstock_amy2018_release_2022_1_metadata."in.ceiling_fan" AS "in.ceiling_fan", resstock_amy2018_release_2022_1_metadata."in.census_division" AS "in.census_division", resstock_amy2018_release_2022_1_metadata."in.census_division_recs" AS "in.census_division_recs", resstock_amy2018_release_2022_1_metadata."in.census_region" AS "in.census_region", resstock_amy2018_release_2022_1_metadata."in.city" AS "in.city", resstock_amy2018_release_2022_1_metadata."in.clothes_dryer" AS "in.clothes_dryer", resstock_amy2018_release_2022_1_metadata."in.clothes_washer" AS "in.clothes_washer", resstock_amy2018_release_2022_1_metadata."in.clothes_washer_presence" AS "in.clothes_washer_presence", resstock_amy2018_release_2022_1_metadata."in.cooking_range" AS "in.cooking_range", resstock_amy2018_release_2022_1_metadata."in.cooling_setpoint" AS "in.cooling_setpoint", resstock_amy2018_release_2022_1_metadata."in.cooling_setpoint_has_offset" AS "in.cooling_setpoint_has_offset", resstock_amy2018_release_2022_1_metadata."in.cooling_setpoint_offset_magnitude" AS "in.cooling_setpoint_offset_magnitude", resstock_amy2018_release_2022_1_metadata."in.cooling_setpoint_offset_period" AS "in.cooling_setpoint_offset_period", resstock_amy2018_release_2022_1_metadata."in.corridor" AS "in.corridor", resstock_amy2018_release_2022_1_metadata."in.county" AS "in.county", resstock_amy2018_release_2022_1_metadata."in.county_and_puma" AS "in.county_and_puma", resstock_amy2018_release_2022_1_metadata."in.dehumidifier" AS "in.dehumidifier", resstock_amy2018_release_2022_1_metadata."in.dishwasher" AS "in.dishwasher", resstock_amy2018_release_2022_1_metadata."in.door_area" AS "in.door_area", resstock_amy2018_release_2022_1_metadata."in.doors" AS "in.doors", resstock_amy2018_release_2022_1_metadata."in.ducts" AS "in.ducts", resstock_amy2018_release_2022_1_metadata."in.eaves" AS "in.eaves", resstock_amy2018_release_2022_1_metadata."in.electric_vehicle" AS "in.electric_vehicle", resstock_amy2018_release_2022_1_metadata."in.emissions_electricity_folders" AS "in.emissions_electricity_folders", resstock_amy2018_release_2022_1_metadata."in.emissions_electricity_units" AS "in.emissions_electricity_units", resstock_amy2018_release_2022_1_metadata."in.emissions_electricity_values_or_filepaths" AS "in.emissions_electricity_values_or_filepaths", resstock_amy2018_release_2022_1_metadata."in.emissions_fossil_fuel_units" AS "in.emissions_fossil_fuel_units", resstock_amy2018_release_2022_1_metadata."in.emissions_fuel_oil_values" AS "in.emissions_fuel_oil_values", resstock_amy2018_release_2022_1_metadata."in.emissions_natural_gas_values" AS "in.emissions_natural_gas_values", resstock_amy2018_release_2022_1_metadata."in.emissions_propane_values" AS "in.emissions_propane_values", resstock_amy2018_release_2022_1_metadata."in.emissions_scenario_names" AS "in.emissions_scenario_names", resstock_amy2018_release_2022_1_metadata."in.emissions_types" AS "in.emissions_types", resstock_amy2018_release_2022_1_metadata."in.emissions_wood_values" AS "in.emissions_wood_values", resstock_amy2018_release_2022_1_metadata."in.federal_poverty_level" AS "in.federal_poverty_level", resstock_amy2018_release_2022_1_metadata."in.generation_and_emissions_assessment_region" AS "in.generation_and_emissions_assessment_region", resstock_amy2018_release_2022_1_metadata."in.geometry_attic_type" AS "in.geometry_attic_type", resstock_amy2018_release_2022_1_metadata."in.geometry_building_horizontal_location_mf" AS "in.geometry_building_horizontal_location_mf", resstock_amy2018_release_2022_1_metadata."in.geometry_building_horizontal_location_sfa" AS "in.geometry_building_horizontal_location_sfa", resstock_amy2018_release_2022_1_metadata."in.geometry_building_level_mf" AS "in.geometry_building_level_mf", resstock_amy2018_release_2022_1_metadata."in.geometry_building_number_units_mf" AS "in.geometry_building_number_units_mf", resstock_amy2018_release_2022_1_metadata."in.geometry_building_number_units_sfa" AS "in.geometry_building_number_units_sfa", resstock_amy2018_release_2022_1_metadata."in.geometry_building_type_acs" AS "in.geometry_building_type_acs", resstock_amy2018_release_2022_1_metadata."in.geometry_building_type_height" AS "in.geometry_building_type_height", resstock_amy2018_release_2022_1_metadata."in.geometry_building_type_recs" AS "in.geometry_building_type_recs", resstock_amy2018_release_2022_1_metadata."in.geometry_floor_area" AS "in.geometry_floor_area", resstock_amy2018_release_2022_1_metadata."in.geometry_floor_area_bin" AS "in.geometry_floor_area_bin", resstock_amy2018_release_2022_1_metadata."in.geometry_foundation_type" AS "in.geometry_foundation_type", resstock_amy2018_release_2022_1_metadata."in.geometry_garage" AS "in.geometry_garage", resstock_amy2018_release_2022_1_metadata."in.geometry_stories" AS "in.geometry_stories", resstock_amy2018_release_2022_1_metadata."in.geometry_stories_low_rise" AS "in.geometry_stories_low_rise", resstock_amy2018_release_2022_1_metadata."in.geometry_story_bin" AS "in.geometry_story_bin", resstock_amy2018_release_2022_1_metadata."in.geometry_wall_exterior_finish" AS "in.geometry_wall_exterior_finish", resstock_amy2018_release_2022_1_metadata."in.geometry_wall_type" AS "in.geometry_wall_type", resstock_amy2018_release_2022_1_metadata."in.has_pv" AS "in.has_pv", resstock_amy2018_release_2022_1_metadata."in.heating_fuel" AS "in.heating_fuel", resstock_amy2018_release_2022_1_metadata."in.heating_setpoint" AS "in.heating_setpoint", resstock_amy2018_release_2022_1_metadata."in.heating_setpoint_has_offset" AS "in.heating_setpoint_has_offset", resstock_amy2018_release_2022_1_metadata."in.heating_setpoint_offset_magnitude" AS "in.heating_setpoint_offset_magnitude", resstock_amy2018_release_2022_1_metadata."in.heating_setpoint_offset_period" AS "in.heating_setpoint_offset_period", resstock_amy2018_release_2022_1_metadata."in.holiday_lighting" AS "in.holiday_lighting", resstock_amy2018_release_2022_1_metadata."in.hot_water_distribution" AS "in.hot_water_distribution", resstock_amy2018_release_2022_1_metadata."in.hot_water_fixtures" AS "in.hot_water_fixtures", resstock_amy2018_release_2022_1_metadata."in.hvac_cooling_efficiency" AS "in.hvac_cooling_efficiency", resstock_amy2018_release_2022_1_metadata."in.hvac_cooling_partial_space_conditioning" AS "in.hvac_cooling_partial_space_conditioning", resstock_amy2018_release_2022_1_metadata."in.hvac_cooling_type" AS "in.hvac_cooling_type", resstock_amy2018_release_2022_1_metadata."in.hvac_has_ducts" AS "in.hvac_has_ducts", resstock_amy2018_release_2022_1_metadata."in.hvac_has_shared_system" AS "in.hvac_has_shared_system", resstock_amy2018_release_2022_1_metadata."in.hvac_has_zonal_electric_heating" AS "in.hvac_has_zonal_electric_heating", resstock_amy2018_release_2022_1_metadata."in.hvac_heating_efficiency" AS "in.hvac_heating_efficiency", resstock_amy2018_release_2022_1_metadata."in.hvac_heating_type" AS "in.hvac_heating_type", resstock_amy2018_release_2022_1_metadata."in.hvac_heating_type_and_fuel" AS "in.hvac_heating_type_and_fuel", resstock_amy2018_release_2022_1_metadata."in.hvac_secondary_heating_efficiency" AS "in.hvac_secondary_heating_efficiency", resstock_amy2018_release_2022_1_metadata."in.hvac_secondary_heating_type_and_fuel" AS "in.hvac_secondary_heating_type_and_fuel", resstock_amy2018_release_2022_1_metadata."in.hvac_shared_efficiencies" AS "in.hvac_shared_efficiencies", resstock_amy2018_release_2022_1_metadata."in.hvac_system_is_faulted" AS "in.hvac_system_is_faulted", resstock_amy2018_release_2022_1_metadata."in.hvac_system_single_speed_ac_airflow" AS "in.hvac_system_single_speed_ac_airflow", resstock_amy2018_release_2022_1_metadata."in.hvac_system_single_speed_ac_charge" AS "in.hvac_system_single_speed_ac_charge", resstock_amy2018_release_2022_1_metadata."in.hvac_system_single_speed_ashp_airflow" AS "in.hvac_system_single_speed_ashp_airflow", resstock_amy2018_release_2022_1_metadata."in.hvac_system_single_speed_ashp_charge" AS "in.hvac_system_single_speed_ashp_charge", resstock_amy2018_release_2022_1_metadata."in.income" AS "in.income", resstock_amy2018_release_2022_1_metadata."in.income_recs_2015" AS "in.income_recs_2015", resstock_amy2018_release_2022_1_metadata."in.income_recs_2020" AS "in.income_recs_2020", resstock_amy2018_release_2022_1_metadata."in.infiltration" AS "in.infiltration", resstock_amy2018_release_2022_1_metadata."in.insulation_ceiling" AS "in.insulation_ceiling", resstock_amy2018_release_2022_1_metadata."in.insulation_floor" AS "in.insulation_floor", resstock_amy2018_release_2022_1_metadata."in.insulation_foundation_wall" AS "in.insulation_foundation_wall", resstock_amy2018_release_2022_1_metadata."in.insulation_rim_joist" AS "in.insulation_rim_joist", resstock_amy2018_release_2022_1_metadata."in.insulation_roof" AS "in.insulation_roof", resstock_amy2018_release_2022_1_metadata."in.insulation_slab" AS "in.insulation_slab", resstock_amy2018_release_2022_1_metadata."in.insulation_wall" AS "in.insulation_wall", resstock_amy2018_release_2022_1_metadata."in.interior_shading" AS "in.interior_shading", resstock_amy2018_release_2022_1_metadata."in.iso_rto_region" AS "in.iso_rto_region", resstock_amy2018_release_2022_1_metadata."in.lighting" AS "in.lighting", resstock_amy2018_release_2022_1_metadata."in.lighting_interior_use" AS "in.lighting_interior_use", resstock_amy2018_release_2022_1_metadata."in.lighting_other_use" AS "in.lighting_other_use", resstock_amy2018_release_2022_1_metadata."in.location_region" AS "in.location_region", resstock_amy2018_release_2022_1_metadata."in.mechanical_ventilation" AS "in.mechanical_ventilation", resstock_amy2018_release_2022_1_metadata."in.misc_extra_refrigerator" AS "in.misc_extra_refrigerator", resstock_amy2018_release_2022_1_metadata."in.misc_freezer" AS "in.misc_freezer", resstock_amy2018_release_2022_1_metadata."in.misc_gas_fireplace" AS "in.misc_gas_fireplace", resstock_amy2018_release_2022_1_metadata."in.misc_gas_grill" AS "in.misc_gas_grill", resstock_amy2018_release_2022_1_metadata."in.misc_gas_lighting" AS "in.misc_gas_lighting", resstock_amy2018_release_2022_1_metadata."in.misc_hot_tub_spa" AS "in.misc_hot_tub_spa", resstock_amy2018_release_2022_1_metadata."in.misc_pool" AS "in.misc_pool", resstock_amy2018_release_2022_1_metadata."in.misc_pool_heater" AS "in.misc_pool_heater", resstock_amy2018_release_2022_1_metadata."in.misc_pool_pump" AS "in.misc_pool_pump", resstock_amy2018_release_2022_1_metadata."in.misc_well_pump" AS "in.misc_well_pump", resstock_amy2018_release_2022_1_metadata."in.natural_ventilation" AS "in.natural_ventilation", resstock_amy2018_release_2022_1_metadata."in.neighbors" AS "in.neighbors", resstock_amy2018_release_2022_1_metadata."in.occupants" AS "in.occupants", resstock_amy2018_release_2022_1_metadata."in.orientation" AS "in.orientation", resstock_amy2018_release_2022_1_metadata."in.overhangs" AS "in.overhangs", resstock_amy2018_release_2022_1_metadata."in.plug_load_diversity" AS "in.plug_load_diversity", resstock_amy2018_release_2022_1_metadata."in.plug_loads" AS "in.plug_loads", resstock_amy2018_release_2022_1_metadata."in.puma" AS "in.puma", resstock_amy2018_release_2022_1_metadata."in.puma_metro_status" AS "in.puma_metro_status", resstock_amy2018_release_2022_1_metadata."in.pv_orientation" AS "in.pv_orientation", resstock_amy2018_release_2022_1_metadata."in.pv_system_size" AS "in.pv_system_size", resstock_amy2018_release_2022_1_metadata."in.radiant_barrier" AS "in.radiant_barrier", resstock_amy2018_release_2022_1_metadata."in.range_spot_vent_hour" AS "in.range_spot_vent_hour", resstock_amy2018_release_2022_1_metadata."in.reeds_balancing_area" AS "in.reeds_balancing_area", resstock_amy2018_release_2022_1_metadata."in.refrigerator" AS "in.refrigerator", resstock_amy2018_release_2022_1_metadata."in.roof_material" AS "in.roof_material", resstock_amy2018_release_2022_1_metadata."in.schedules" AS "in.schedules", resstock_amy2018_release_2022_1_metadata."in.simulation_control_run_period_begin_day_of_month" AS "in.simulation_control_run_period_begin_day_of_month", resstock_amy2018_release_2022_1_metadata."in.simulation_control_run_period_begin_month" AS "in.simulation_control_run_period_begin_month", resstock_amy2018_release_2022_1_metadata."in.simulation_control_run_period_calendar_year" AS "in.simulation_control_run_period_calendar_year", resstock_amy2018_release_2022_1_metadata."in.simulation_control_run_period_end_day_of_month" AS "in.simulation_control_run_period_end_day_of_month", resstock_amy2018_release_2022_1_metadata."in.simulation_control_run_period_end_month" AS "in.simulation_control_run_period_end_month", resstock_amy2018_release_2022_1_metadata."in.simulation_control_timestep" AS "in.simulation_control_timestep", resstock_amy2018_release_2022_1_metadata."in.solar_hot_water" AS "in.solar_hot_water", resstock_amy2018_release_2022_1_metadata."in.state" AS "in.state", resstock_amy2018_release_2022_1_metadata."in.tenure" AS "in.tenure", resstock_amy2018_release_2022_1_metadata."in.units_represented" AS "in.units_represented", resstock_amy2018_release_2022_1_metadata."in.usage_level" AS "in.usage_level", resstock_amy2018_release_2022_1_metadata."in.vacancy_status" AS "in.vacancy_status", resstock_amy2018_release_2022_1_metadata."in.vintage" AS "in.vintage", resstock_amy2018_release_2022_1_metadata."in.vintage_acs" AS "in.vintage_acs", resstock_amy2018_release_2022_1_metadata."in.water_heater_efficiency" AS "in.water_heater_efficiency", resstock_amy2018_release_2022_1_metadata."in.water_heater_fuel" AS "in.water_heater_fuel", resstock_amy2018_release_2022_1_metadata."in.water_heater_in_unit" AS "in.water_heater_in_unit", resstock_amy2018_release_2022_1_metadata."in.weather_file_city" AS "in.weather_file_city", resstock_amy2018_release_2022_1_metadata."in.weather_file_latitude" AS "in.weather_file_latitude", resstock_amy2018_release_2022_1_metadata."in.weather_file_longitude" AS "in.weather_file_longitude", resstock_amy2018_release_2022_1_metadata."in.window_areas" AS "in.window_areas", resstock_amy2018_release_2022_1_metadata."in.windows" AS "in.windows", resstock_amy2018_release_2022_1_metadata."out.electricity.total.lrmer_95_decarb_by_2035_15_2025_start.co2e_kg" AS "out.electricity.total.lrmer_95_decarb_by_2035_15_2025_start.co2e_kg", resstock_amy2018_release_2022_1_metadata."out.fuel_oil.total.lrmer_95_decarb_by_2035_15_2025_start.co2e_kg" AS "out.fuel_oil.total.lrmer_95_decarb_by_2035_15_2025_start.co2e_kg", resstock_amy2018_release_2022_1_metadata."out.natural_gas.total.lrmer_95_decarb_by_2035_15_2025_start.co2e_kg" AS "out.natural_gas.total.lrmer_95_decarb_by_2035_15_2025_start.co2e_kg", resstock_amy2018_release_2022_1_metadata."out.propane.total.lrmer_95_decarb_by_2035_15_2025_start.co2e_kg" AS "out.propane.total.lrmer_95_decarb_by_2035_15_2025_start.co2e_kg", resstock_amy2018_release_2022_1_metadata."out.electricity.total.lrmer_low_re_cost_15_2025_start.co2e_kg" AS "out.electricity.total.lrmer_low_re_cost_15_2025_start.co2e_kg", resstock_amy2018_release_2022_1_metadata."out.fuel_oil.total.lrmer_low_re_cost_15_2025_start.co2e_kg" AS "out.fuel_oil.total.lrmer_low_re_cost_15_2025_start.co2e_kg", resstock_amy2018_release_2022_1_metadata."out.natural_gas.total.lrmer_low_re_cost_15_2025_start.co2e_kg" AS "out.natural_gas.total.lrmer_low_re_cost_15_2025_start.co2e_kg", resstock_amy2018_release_2022_1_metadata."out.propane.total.lrmer_low_re_cost_15_2025_start.co2e_kg" AS "out.propane.total.lrmer_low_re_cost_15_2025_start.co2e_kg", resstock_amy2018_release_2022_1_metadata."out.electricity.total.lrmer_low_re_cost_25_2025_start.co2e_kg" AS "out.electricity.total.lrmer_low_re_cost_25_2025_start.co2e_kg", resstock_amy2018_release_2022_1_metadata."out.fuel_oil.total.lrmer_low_re_cost_25_2025_start.co2e_kg" AS "out.fuel_oil.total.lrmer_low_re_cost_25_2025_start.co2e_kg", resstock_amy2018_release_2022_1_metadata."out.natural_gas.total.lrmer_low_re_cost_25_2025_start.co2e_kg" AS "out.natural_gas.total.lrmer_low_re_cost_25_2025_start.co2e_kg", resstock_amy2018_release_2022_1_metadata."out.propane.total.lrmer_low_re_cost_25_2025_start.co2e_kg" AS "out.propane.total.lrmer_low_re_cost_25_2025_start.co2e_kg", resstock_amy2018_release_2022_1_metadata."out.electricity.total.lrmer_mid_case_15_2025_start.co2e_kg" AS "out.electricity.total.lrmer_mid_case_15_2025_start.co2e_kg", resstock_amy2018_release_2022_1_metadata."out.fuel_oil.total.lrmer_mid_case_15_2025_start.co2e_kg" AS "out.fuel_oil.total.lrmer_mid_case_15_2025_start.co2e_kg", resstock_amy2018_release_2022_1_metadata."out.natural_gas.total.lrmer_mid_case_15_2025_start.co2e_kg" AS "out.natural_gas.total.lrmer_mid_case_15_2025_start.co2e_kg", resstock_amy2018_release_2022_1_metadata."out.propane.total.lrmer_mid_case_15_2025_start.co2e_kg" AS "out.propane.total.lrmer_mid_case_15_2025_start.co2e_kg", resstock_amy2018_release_2022_1_metadata."out.electricity.ceiling_fan.energy_consumption" AS "out.electricity.ceiling_fan.energy_consumption", resstock_amy2018_release_2022_1_metadata."out.electricity.ceiling_fan.energy_consumption_intensity" AS "out.electricity.ceiling_fan.energy_consumption_intensity", resstock_amy2018_release_2022_1_metadata."out.electricity.clothes_dryer.energy_consumption" AS "out.electricity.clothes_dryer.energy_consumption", resstock_amy2018_release_2022_1_metadata."out.electricity.clothes_dryer.energy_consumption_intensity" AS "out.electricity.clothes_dryer.energy_consumption_intensity", resstock_amy2018_release_2022_1_metadata."out.electricity.clothes_washer.energy_consumption" AS "out.electricity.clothes_washer.energy_consumption", resstock_amy2018_release_2022_1_metadata."out.electricity.clothes_washer.energy_consumption_intensity" AS "out.electricity.clothes_washer.energy_consumption_intensity", resstock_amy2018_release_2022_1_metadata."out.electricity.cooling_fans_pumps.energy_consumption" AS "out.electricity.cooling_fans_pumps.energy_consumption", resstock_amy2018_release_2022_1_metadata."out.electricity.cooling_fans_pumps.energy_consumption_intensity" AS "out.electricity.cooling_fans_pumps.energy_consumption_intensity", resstock_amy2018_release_2022_1_metadata."out.electricity.cooling.energy_consumption" AS "out.electricity.cooling.energy_consumption", resstock_amy2018_release_2022_1_metadata."out.electricity.cooling.energy_consumption_intensity" AS "out.electricity.cooling.energy_consumption_intensity", resstock_amy2018_release_2022_1_metadata."out.electricity.dishwasher.energy_consumption" AS "out.electricity.dishwasher.energy_consumption", resstock_amy2018_release_2022_1_metadata."out.electricity.dishwasher.energy_consumption_intensity" AS "out.electricity.dishwasher.energy_consumption_intensity", resstock_amy2018_release_2022_1_metadata."out.electricity.freezer.energy_consumption" AS "out.electricity.freezer.energy_consumption", resstock_amy2018_release_2022_1_metadata."out.electricity.freezer.energy_consumption_intensity" AS "out.electricity.freezer.energy_consumption_intensity", resstock_amy2018_release_2022_1_metadata."out.electricity.heating_fans_pumps.energy_consumption" AS "out.electricity.heating_fans_pumps.energy_consumption", resstock_amy2018_release_2022_1_metadata."out.electricity.heating_fans_pumps.energy_consumption_intensity" AS "out.electricity.heating_fans_pumps.energy_consumption_intensity", resstock_amy2018_release_2022_1_metadata."out.electricity.heating_hp_bkup.energy_consumption" AS "out.electricity.heating_hp_bkup.energy_consumption", resstock_amy2018_release_2022_1_metadata."out.electricity.heating_hp_bkup.energy_consumption_intensity" AS "out.electricity.heating_hp_bkup.energy_consumption_intensity", resstock_amy2018_release_2022_1_metadata."out.electricity.heating.energy_consumption" AS "out.electricity.heating.energy_consumption", resstock_amy2018_release_2022_1_metadata."out.electricity.heating.energy_consumption_intensity" AS "out.electricity.heating.energy_consumption_intensity", resstock_amy2018_release_2022_1_metadata."out.electricity.hot_tub_heater.energy_consumption" AS "out.electricity.hot_tub_heater.energy_consumption", resstock_amy2018_release_2022_1_metadata."out.electricity.hot_tub_heater.energy_consumption_intensity" AS "out.electricity.hot_tub_heater.energy_consumption_intensity", resstock_amy2018_release_2022_1_metadata."out.electricity.hot_tub_pump.energy_consumption" AS "out.electricity.hot_tub_pump.energy_consumption", resstock_amy2018_release_2022_1_metadata."out.electricity.hot_tub_pump.energy_consumption_intensity" AS "out.electricity.hot_tub_pump.energy_consumption_intensity", resstock_amy2018_release_2022_1_metadata."out.electricity.hot_water.energy_consumption" AS "out.electricity.hot_water.energy_consumption", resstock_amy2018_release_2022_1_metadata."out.electricity.hot_water.energy_consumption_intensity" AS "out.electricity.hot_water.energy_consumption_intensity", resstock_amy2018_release_2022_1_metadata."out.electricity.lighting_exterior.energy_consumption" AS "out.electricity.lighting_exterior.energy_consumption", resstock_amy2018_release_2022_1_metadata."out.electricity.lighting_exterior.energy_consumption_intensity" AS "out.electricity.lighting_exterior.energy_consumption_intensity", resstock_amy2018_release_2022_1_metadata."out.electricity.lighting_garage.energy_consumption" AS "out.electricity.lighting_garage.energy_consumption", resstock_amy2018_release_2022_1_metadata."out.electricity.lighting_garage.energy_consumption_intensity" AS "out.electricity.lighting_garage.energy_consumption_intensity", resstock_amy2018_release_2022_1_metadata."out.electricity.lighting_interior.energy_consumption" AS "out.electricity.lighting_interior.energy_consumption", resstock_amy2018_release_2022_1_metadata."out.electricity.lighting_interior.energy_consumption_intensity" AS "out.electricity.lighting_interior.energy_consumption_intensity", resstock_amy2018_release_2022_1_metadata."out.electricity.mech_vent.energy_consumption" AS "out.electricity.mech_vent.energy_consumption", resstock_amy2018_release_2022_1_metadata."out.electricity.mech_vent.energy_consumption_intensity" AS "out.electricity.mech_vent.energy_consumption_intensity", resstock_amy2018_release_2022_1_metadata."out.electricity.plug_loads.energy_consumption" AS "out.electricity.plug_loads.energy_consumption", resstock_amy2018_release_2022_1_metadata."out.electricity.plug_loads.energy_consumption_intensity" AS "out.electricity.plug_loads.energy_consumption_intensity", resstock_amy2018_release_2022_1_metadata."out.electricity.pool_heater.energy_consumption" AS "out.electricity.pool_heater.energy_consumption", resstock_amy2018_release_2022_1_metadata."out.electricity.pool_heater.energy_consumption_intensity" AS "out.electricity.pool_heater.energy_consumption_intensity", resstock_amy2018_release_2022_1_metadata."out.electricity.pool_pump.energy_consumption" AS "out.electricity.pool_pump.energy_consumption", resstock_amy2018_release_2022_1_metadata."out.electricity.pool_pump.energy_consumption_intensity" AS "out.electricity.pool_pump.energy_consumption_intensity", resstock_amy2018_release_2022_1_metadata."out.electricity.pv.energy_consumption" AS "out.electricity.pv.energy_consumption", resstock_amy2018_release_2022_1_metadata."out.electricity.pv.energy_consumption_intensity" AS "out.electricity.pv.energy_consumption_intensity", resstock_amy2018_release_2022_1_metadata."out.electricity.range_oven.energy_consumption" AS "out.electricity.range_oven.energy_consumption", resstock_amy2018_release_2022_1_metadata."out.electricity.range_oven.energy_consumption_intensity" AS "out.electricity.range_oven.energy_consumption_intensity", resstock_amy2018_release_2022_1_metadata."out.electricity.refrigerator.energy_consumption" AS "out.electricity.refrigerator.energy_consumption", resstock_amy2018_release_2022_1_metadata."out.electricity.refrigerator.energy_consumption_intensity" AS "out.electricity.refrigerator.energy_consumption_intensity", resstock_amy2018_release_2022_1_metadata."out.electricity.well_pump.energy_consumption" AS "out.electricity.well_pump.energy_consumption", resstock_amy2018_release_2022_1_metadata."out.electricity.well_pump.energy_consumption_intensity" AS "out.electricity.well_pump.energy_consumption_intensity", resstock_amy2018_release_2022_1_metadata."out.fuel_oil.heating_hp_bkup.energy_consumption" AS "out.fuel_oil.heating_hp_bkup.energy_consumption", resstock_amy2018_release_2022_1_metadata."out.fuel_oil.heating_hp_bkup.energy_consumption_intensity" AS "out.fuel_oil.heating_hp_bkup.energy_consumption_intensity", resstock_amy2018_release_2022_1_metadata."out.fuel_oil.heating.energy_consumption" AS "out.fuel_oil.heating.energy_consumption", resstock_amy2018_release_2022_1_metadata."out.fuel_oil.heating.energy_consumption_intensity" AS "out.fuel_oil.heating.energy_consumption_intensity", resstock_amy2018_release_2022_1_metadata."out.fuel_oil.hot_water.energy_consumption" AS "out.fuel_oil.hot_water.energy_consumption", resstock_amy2018_release_2022_1_metadata."out.fuel_oil.hot_water.energy_consumption_intensity" AS "out.fuel_oil.hot_water.energy_consumption_intensity", resstock_amy2018_release_2022_1_metadata."out.natural_gas.clothes_dryer.energy_consumption" AS "out.natural_gas.clothes_dryer.energy_consumption", resstock_amy2018_release_2022_1_metadata."out.natural_gas.clothes_dryer.energy_consumption_intensity" AS "out.natural_gas.clothes_dryer.energy_consumption_intensity", resstock_amy2018_release_2022_1_metadata."out.natural_gas.fireplace.energy_consumption" AS "out.natural_gas.fireplace.energy_consumption", resstock_amy2018_release_2022_1_metadata."out.natural_gas.fireplace.energy_consumption_intensity" AS "out.natural_gas.fireplace.energy_consumption_intensity", resstock_amy2018_release_2022_1_metadata."out.natural_gas.grill.energy_consumption" AS "out.natural_gas.grill.energy_consumption", resstock_amy2018_release_2022_1_metadata."out.natural_gas.grill.energy_consumption_intensity" AS "out.natural_gas.grill.energy_consumption_intensity", resstock_amy2018_release_2022_1_metadata."out.natural_gas.heating_hp_bkup.energy_consumption" AS "out.natural_gas.heating_hp_bkup.energy_consumption", resstock_amy2018_release_2022_1_metadata."out.natural_gas.heating_hp_bkup.energy_consumption_intensity" AS "out.natural_gas.heating_hp_bkup.energy_consumption_intensity", resstock_amy2018_release_2022_1_metadata."out.natural_gas.heating.energy_consumption" AS "out.natural_gas.heating.energy_consumption", resstock_amy2018_release_2022_1_metadata."out.natural_gas.heating.energy_consumption_intensity" AS "out.natural_gas.heating.energy_consumption_intensity", resstock_amy2018_release_2022_1_metadata."out.natural_gas.hot_tub_heater.energy_consumption" AS "out.natural_gas.hot_tub_heater.energy_consumption", resstock_amy2018_release_2022_1_metadata."out.natural_gas.hot_tub_heater.energy_consumption_intensity" AS "out.natural_gas.hot_tub_heater.energy_consumption_intensity", resstock_amy2018_release_2022_1_metadata."out.natural_gas.hot_water.energy_consumption" AS "out.natural_gas.hot_water.energy_consumption", resstock_amy2018_release_2022_1_metadata."out.natural_gas.hot_water.energy_consumption_intensity" AS "out.natural_gas.hot_water.energy_consumption_intensity", resstock_amy2018_release_2022_1_metadata."out.natural_gas.lighting.energy_consumption" AS "out.natural_gas.lighting.energy_consumption", resstock_amy2018_release_2022_1_metadata."out.natural_gas.lighting.energy_consumption_intensity" AS "out.natural_gas.lighting.energy_consumption_intensity", resstock_amy2018_release_2022_1_metadata."out.natural_gas.pool_heater.energy_consumption" AS "out.natural_gas.pool_heater.energy_consumption", resstock_amy2018_release_2022_1_metadata."out.natural_gas.pool_heater.energy_consumption_intensity" AS "out.natural_gas.pool_heater.energy_consumption_intensity", resstock_amy2018_release_2022_1_metadata."out.natural_gas.range_oven.energy_consumption" AS "out.natural_gas.range_oven.energy_consumption", resstock_amy2018_release_2022_1_metadata."out.natural_gas.range_oven.energy_consumption_intensity" AS "out.natural_gas.range_oven.energy_consumption_intensity", resstock_amy2018_release_2022_1_metadata."out.propane.clothes_dryer.energy_consumption" AS "out.propane.clothes_dryer.energy_consumption", resstock_amy2018_release_2022_1_metadata."out.propane.clothes_dryer.energy_consumption_intensity" AS "out.propane.clothes_dryer.energy_consumption_intensity", resstock_amy2018_release_2022_1_metadata."out.propane.heating_hp_bkup.energy_consumption" AS "out.propane.heating_hp_bkup.energy_consumption", resstock_amy2018_release_2022_1_metadata."out.propane.heating_hp_bkup.energy_consumption_intensity" AS "out.propane.heating_hp_bkup.energy_consumption_intensity", resstock_amy2018_release_2022_1_metadata."out.propane.heating.energy_consumption" AS "out.propane.heating.energy_consumption", resstock_amy2018_release_2022_1_metadata."out.propane.heating.energy_consumption_intensity" AS "out.propane.heating.energy_consumption_intensity", resstock_amy2018_release_2022_1_metadata."out.propane.hot_water.energy_consumption" AS "out.propane.hot_water.energy_consumption", resstock_amy2018_release_2022_1_metadata."out.propane.hot_water.energy_consumption_intensity" AS "out.propane.hot_water.energy_consumption_intensity", resstock_amy2018_release_2022_1_metadata."out.propane.range_oven.energy_consumption" AS "out.propane.range_oven.energy_consumption", resstock_amy2018_release_2022_1_metadata."out.propane.range_oven.energy_consumption_intensity" AS "out.propane.range_oven.energy_consumption_intensity", resstock_amy2018_release_2022_1_metadata."out.site_energy.net.energy_consumption" AS "out.site_energy.net.energy_consumption", resstock_amy2018_release_2022_1_metadata."out.site_energy.net.energy_consumption_intensity" AS "out.site_energy.net.energy_consumption_intensity", resstock_amy2018_release_2022_1_metadata."out.site_energy.total.energy_consumption" AS "out.site_energy.total.energy_consumption", resstock_amy2018_release_2022_1_metadata."out.site_energy.total.energy_consumption_intensity" AS "out.site_energy.total.energy_consumption_intensity", resstock_amy2018_release_2022_1_metadata."out.electricity.net.energy_consumption" AS "out.electricity.net.energy_consumption", resstock_amy2018_release_2022_1_metadata."out.electricity.net.energy_consumption_intensity" AS "out.electricity.net.energy_consumption_intensity", resstock_amy2018_release_2022_1_metadata."out.electricity.total.energy_consumption" AS "out.electricity.total.energy_consumption", resstock_amy2018_release_2022_1_metadata."out.electricity.total.energy_consumption_intensity" AS "out.electricity.total.energy_consumption_intensity", resstock_amy2018_release_2022_1_metadata."out.fuel_oil.total.energy_consumption" AS "out.fuel_oil.total.energy_consumption", resstock_amy2018_release_2022_1_metadata."out.fuel_oil.total.energy_consumption_intensity" AS "out.fuel_oil.total.energy_consumption_intensity", resstock_amy2018_release_2022_1_metadata."out.natural_gas.total.energy_consumption" AS "out.natural_gas.total.energy_consumption", resstock_amy2018_release_2022_1_metadata."out.natural_gas.total.energy_consumption_intensity" AS "out.natural_gas.total.energy_consumption_intensity", resstock_amy2018_release_2022_1_metadata."out.propane.total.energy_consumption" AS "out.propane.total.energy_consumption", resstock_amy2018_release_2022_1_metadata."out.propane.total.energy_consumption_intensity" AS "out.propane.total.energy_consumption_intensity", resstock_amy2018_release_2022_1_metadata.upgrade AS upgrade, resstock_amy2018_release_2022_1_metadata.metadata_index AS metadata_index, resstock_amy2018_release_2022_1_metadata.bldg_id AS bldg_id 
FROM resstock_amy2018_release_2022_1_metadata 
WHERE CAST(resstock_amy2018_release_2022_1_metadata.upgrade AS VARCHAR) = '0') AS baseline 
WHERE CAST(baseline.applicability AS VARCHAR) = 'true' GROUP BY 1 ORDER BY 1
""")
baseline_agg

,geometry_building_type_recs,sample_count,units_count,electricitytotalenergy_consumption
0,Mobile Home,34491,8.351341e+06,9.934359e+10
1,Multi-Family with 2 - 4 Units,43868,1.062180e+07,8.623094e+10
2,Multi-Family with 5+ Units,100345,2.429664e+07,1.769924e+11
3,Single-Family Attached,32125,7.778459e+06,7.379245e+10
4,Single-Family Detached,338087,8.186135e+07,1.115071e+12


In [13]:
baseline_agg_query2 = my_run.query(enduses=['electricity.total.energy_consumption'],
                                       group_by=['geometry_building_type_recs'],
                                          get_query_only=True)
print(baseline_agg_query2)

SELECT baseline."in.geometry_building_type_recs" AS geometry_building_type_recs, sum(1) AS sample_count, sum(baseline.weight) AS units_count, sum(baseline."out.electricity.total.energy_consumption" * baseline.weight) AS "electricity.total.energy_consumption" 
FROM (SELECT resstock_amy2018_release_2022_1_metadata.applicability AS applicability, resstock_amy2018_release_2022_1_metadata."in.sqft" AS "in.sqft", resstock_amy2018_release_2022_1_metadata.weight AS weight, resstock_amy2018_release_2022_1_metadata."in.ahs_region" AS "in.ahs_region", resstock_amy2018_release_2022_1_metadata."in.ashrae_iecc_climate_zone_2004" AS "in.ashrae_iecc_climate_zone_2004", resstock_amy2018_release_2022_1_metadata."in.ashrae_iecc_climate_zone_2004_2_a_split" AS "in.ashrae_iecc_climate_zone_2004_2_a_split", resstock_amy2018_release_2022_1_metadata."in.bathroom_spot_vent_hour" AS "in.bathroom_spot_vent_hour", resstock_amy2018_release_2022_1_metadata."in.bedrooms" AS "in.bedrooms", resstock_amy2018_release_20

In [14]:
bq = my_run.submit_batch_query([baseline_agg_query, baseline_agg_query2])
print(bq)
my_run.get_batch_query_report(bq)

INFO:buildstock_query.query_core:Submitted queries[0] (CACHED)


2


INFO:buildstock_query.query_core:Submitted queries[1] (CACHED)


{'submitted': 0, 'running': 0, 'pending': 2, 'completed': 0, 'failed': 0}

In [15]:
my_run.get_batch_query_report(bq)

{'submitted': 2, 'running': 0, 'pending': 0, 'completed': 2, 'failed': 0}

In [16]:
result_df = my_run.get_batch_query_result(bq, combine=True)
result_df

INFO:buildstock_query.query_core:{'submitted': 2, 'running': 0, 'pending': 0, 'completed': 2, 'failed': 0}
INFO:buildstock_query.query_core:Batch query completed. 
INFO:buildstock_query.query_core:Got result from Query [0] (CACHED)
INFO:buildstock_query.query_core:Got result from Query [1] (CACHED)
INFO:buildstock_query.query_core:Concatenating the results.


,geometry_building_type_recs,sample_count,units_count,electricity.total.energy_consumption,query_id
0,Mobile Home,34491,8.351341e+06,9.934359e+10,0
1,Multi-Family with 2 - 4 Units,43868,1.062180e+07,8.623094e+10,0
2,Multi-Family with 5+ Units,100345,2.429664e+07,1.769924e+11,0
3,Single-Family Attached,32125,7.778459e+06,7.379245e+10,0
4,Single-Family Detached,338087,8.186135e+07,1.115071e+12,0
0,Mobile Home,34491,8.351341e+06,9.934359e+10,1
1,Multi-Family with 2 - 4 Units,43868,1.062180e+07,8.623094e+10,1
2,Multi-Family with 5+ Units,100345,2.429664e+07,1.769924e+11,1
3,Single-Family Attached,32125,7.778459e+06,7.379245e+10,1
4,Single-Family Detached,338087,8.186135e+07,1.115071e+12,1


In [17]:
query1 = my_run.query(enduses=['electricity.total.energy_consumption'], group_by=['state'],
                                              get_query_only=True)
query2 = my_run.query(enduses=['electricity.total.energy_consumption'],
                                              group_by=['state'],
                                              get_query_only=True)
exe_id1, future1 = my_run.execute(query1, run_async=True)
exe_id2, future2 = my_run.execute(query2, run_async=True)

assert exe_id1 != "CACHED"  # Commenting this line will cause the type checker to complain on the next line
my_run.get_query_output_location(exe_id1)

's3://resstock-core/athena_query_results/26e9c80b-4d6c-4dc0-b2ca-0d8106c4ad3b.csv'

In [18]:
future1.as_pandas()

ERROR:concurrent.futures:exception calling callback for <Future at 0x15e9f16c0 state=finished returned AthenaPandasResultSet>
Traceback (most recent call last):
  File "/Users/radhikar/.local/share/uv/python/cpython-3.10.18-macos-aarch64-none/lib/python3.10/concurrent/futures/_base.py", line 342, in _invoke_callbacks
    callback(self)
  File "/Users/radhikar/Documents/buildstock2025/buildstock-query-applied_only/buildstock_query/query_core.py", line 561, in <lambda>
    result_future.add_done_callback(lambda f: self._query_cache.update({query: f.as_pandas()}))
  File "/Users/radhikar/Documents/buildstock2025/buildstock-query-applied_only/buildstock_query/query_core.py", line 555, in get_pandas
    raise OperationalError(f"{res.state}: {res.state_change_reason}")
pyathena.error.OperationalError: FAILED: Query exhausted resources at this scale factor
ERROR:concurrent.futures:exception calling callback for <Future at 0x1527dd840 state=finished returned AthenaPandasResultSet>
Traceback (m

OperationalError: FAILED: Query exhausted resources at this scale factor

In [26]:
my_run.get_query_status(ExeId(exe_id1))

'SUCCEEDED'

In [27]:
# my_run.get_athena_query_result(exe_id1)


In [28]:
my_run.get_all_running_queries()

[]

In [29]:
my_run.stop_all_queries()

INFO:buildstock_query.query_core:Stopped 0 queries


In [22]:
my_run.stop_query(ExeId(exe_id1))

{'ResponseMetadata': {'RequestId': '3f6d2809-1ff7-46ba-a455-dd3cdfbc54fb',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 14 Apr 2023 15:56:16 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': '3f6d2809-1ff7-46ba-a455-dd3cdfbc54fb'},
  'RetryAttempts': 0}}

In [23]:
my_run.delete_everything()

Will delete the following tables ['res_test_03_2018_10k_20220607_baseline', 'res_test_03_2018_10k_20220607_timeseries', 'res_test_03_2018_10k_20220607_upgrades'] and the s3:/euss/tests/res_test_03_2018_10k_20220607 folder
Abandoned the idea.
